In [39]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [40]:
from pyspark import SparkContext
from pyspark.sql.functions import explode
import csv
import sys
import fiona
import fiona.crs
import shapely
import rtree
import pandas as pd
import geopandas as gpd
from functools import partial
import pyproj
pyproj.__file__

'C:\\Users\\sonali\\Anaconda3\\lib\\site-packages\\pyproj\\__init__.py'

In [41]:
import os
os.environ["PROJ_LIB"] = 'C:\\Users\\sonali\\Anaconda3\\Library\\share'

In [42]:
tweets = 'tweets-100m.txt'
cities = '500cities_tracts.geojson'
drugs = 'drug_*.txt'
#drugs1 = "drug_illegal.txt"
#drugs2 = "drug_sched2.txt"

In [43]:
tweets = sc.textFile(tweets,use_unicode=True).cache()
tweets.take(5)

['450845003775234048|35.82840584|-78.58814956|MarkaelDupree|Tue Apr 01 03:59:59 +0000 2014|@RatedRnasty3_ haha hopefully !|haha hopefully',
 '450845003854536704|32.80947163|-116.91216669|darthraedar|Tue Apr 01 03:59:59 +0000 2014|@kenraldchua SAME|same',
 '450845003896479744|33.01608281|-97.30442766|hungrypoop|Tue Apr 01 03:59:59 +0000 2014|i wish i could play the moog synthesizer|could moog play synthesizer the wish',
 "450845003904868354|41.61400034|-87.91314315|Socalfan21|Tue Apr 01 03:59:59 +0000 2014|@ChiBulls_Chris we can rewrite a script and reenact the show if you'd like.|and can like reenact rewrite script show the you",
 '450845003925843968|30.1110886|-95.12464941|NikeZROking|Tue Apr 01 03:59:59 +0000 2014|@KKalbitz haha. Your dms are going to shut down twitter after tonight|after are dms down going haha shut tonight twitter your']

In [44]:
keywords = sc.textFile(drugs, use_unicode=True).cache()
keywords.take(5)

['abyssinian tea', 'acid', 'adam', 'african salad', 'alcohol']

In [45]:
keywords = set(sc.textFile(drugs, use_unicode=True).cache().collect())
drugterms = sc.broadcast(keywords)
drugterms.value

{'chocolate chip cookies',
 'differin cream',
 'astagraf xl',
 'cresemba',
 'aldara',
 'dutoprol',
 'tev tropin',
 'creon 20',
 'peridex',
 'levetiracetam',
 'ddavp nasal spray',
 'mandol',
 'zocor',
 'xalatan',
 'fluvirin',
 'nasalcrom',
 'buphenyl',
 'elavil',
 'dilantin',
 'mvi adult',
 'heparin',
 'genie',
 'pseudovent 400',
 'xermelo',
 'myozyme',
 'zenzedi',
 'imbruvica',
 'deconex dm',
 'epanova',
 'fluoroplex',
 'dimetane',
 'soriatane',
 'niferex gold tablets',
 'glatopa',
 'ibrance',
 'peace pill',
 'cat',
 'flarex',
 'benz',
 'abraxane',
 'kerledex',
 'crofab',
 'moviprep',
 'novothyrox',
 'dovonex ointment',
 'depocyt',
 'supprelin la',
 'requip',
 'mepron',
 'nutrestore',
 'prudoxin',
 'lartruvo',
 'roofies',
 'triple c',
 'clozaril',
 'feiba vh',
 'weight gainers',
 'boat',
 'lamictal xr',
 'foradil certihaler',
 'esomeprazole magnesium',
 'velphoro',
 'eucrisa',
 'corphedra',
 'herb',
 'black beauties',
 'celontin',
 'smoflipid',
 'xolair',
 'demser',
 'denavir',
 'augme

In [46]:
def countWords(_, rows):
    for row in rows:
        words = row.split()
        yield len(words), 1

words = sc.textFile(drugs, use_unicode=True).cache()\
           .mapPartitionsWithIndex(countWords).reduceByKey(lambda x, y: x+y).collect()

In [47]:
words

[(3, 175), (6, 6), (1, 2611), (4, 37), (7, 1), (2, 763), (5, 16), (8, 1)]

In [48]:
def wordList(_, rows):
    import re
    separator = re.compile('\W+')
    n = 8   #max of 8 words
    
    for row in rows:
        count = []
        field = row.split('|')  
        words = separator.split(field[5].lower())  # text last second column
        length = len(words)

        # listwords
        for i in range(length):
            for j in range(1, n+1):
                if i+j > length:
                    break    # stop when the max has reached
                count.append(' '.join(words[i:i+j])) #join words
        
        # return an array of phrases
        yield ((float(field[2]), float(field[1])), count)

tweetlist = tweets.mapPartitionsWithIndex(wordList)
tweetlist.take(3)

[((-78.58814956, 35.82840584),
  ['',
   ' ratedrnasty3_',
   ' ratedrnasty3_ haha',
   ' ratedrnasty3_ haha hopefully',
   ' ratedrnasty3_ haha hopefully ',
   'ratedrnasty3_',
   'ratedrnasty3_ haha',
   'ratedrnasty3_ haha hopefully',
   'ratedrnasty3_ haha hopefully ',
   'haha',
   'haha hopefully',
   'haha hopefully ',
   'hopefully',
   'hopefully ',
   '']),
 ((-116.91216669, 32.80947163),
  ['',
   ' kenraldchua',
   ' kenraldchua same',
   'kenraldchua',
   'kenraldchua same',
   'same']),
 ((-97.30442766, 33.01608281),
  ['i',
   'i wish',
   'i wish i',
   'i wish i could',
   'i wish i could play',
   'i wish i could play the',
   'i wish i could play the moog',
   'i wish i could play the moog synthesizer',
   'wish',
   'wish i',
   'wish i could',
   'wish i could play',
   'wish i could play the',
   'wish i could play the moog',
   'wish i could play the moog synthesizer',
   'i',
   'i could',
   'i could play',
   'i could play the',
   'i could play the moog',
   

In [49]:
def filterTweets(drugterms, _, rows):
    terms = drugterms.value
    for row in rows:
        log_lat, countlist = row
        match = terms.intersection(countlist)
        if len(match) > 0:
            yield(log_lat, match)
            
drugTweets = tweetlist.mapPartitionsWithIndex(partial(filterTweets, drugterms))
drugTweets.take(5)

[((-117.88537093, 33.59660902), {'boat'}),
 ((-100.29277778, 25.63861111), {'c', 'h'}),
 ((-74.02517944, 40.21185001), {'weed'}),
 ((-78.80715424, 43.00647335), {'sherm'}),
 ((-80.1286475, 25.96150019), {'friend'})]

In [50]:
def filterdrugs(drugterms, _, rows):
    import re
    separator = re.compile('\W+')
    terms = drugterms.value
    for row in rows:
        field = row.split('|')
        words = separator.split(field[-1].lower())
        if len(terms.intersection(words)) > 0:
            yield(float(field[2]), float(field[1]), words)

druglist = tweets.mapPartitionsWithIndex(partial(filterdrugs, drugterms))
druglist.take(5)

[(-117.88537093,
  33.59660902,
  ['boat',
   'bottom',
   'can',
   'cuz',
   'dads',
   'dick',
   'down',
   'drinking',
   'float',
   'heads',
   'ill',
   'lemon',
   'nachos',
   'sangria',
   'the',
   'were',
   'while',
   'won']),
 (-74.02517944, 40.21185001, ['ass', 'hell', 'mad', 'this', 'weed']),
 (-78.80715424, 43.00647335, ['jealous', 'just', 'sherm']),
 (-80.1286475,
  25.96150019,
  ['all',
   'beautiful',
   'birthday',
   'celebrate',
   'come',
   'dreams',
   'fri',
   'friend',
   'happy',
   'lots',
   'love',
   'thurs',
   'true',
   'wed',
   'will',
   'wishing',
   'you',
   'your']),
 (-78.66453529,
  43.21233625,
  ['done', 'got', 'half', 'homework', 'least', 'tonight', 'well'])]

In [51]:
zones = gpd.read_file(cities).to_crs(fiona.crs.from_epsg(2263))
zones.geometry[1].is_valid
zones.columns

Index(['plctract10', 'plctrpop10', 'geometry'], dtype='object')

In [52]:
zones.head(5)

,plctract10,plctrpop10,geometry
0,0107000-01073005910,4612,(POLYGON ((-2901259.247383744 -2109114.1476620...
1,0107000-01073010801,168,(POLYGON ((-2916943.229076727 -2144480.3017811...
2,0107000-01073012701,44,(POLYGON ((-2884835.522685619 -2145459.4804165...
3,0107000-01073012703,498,(POLYGON ((-2884278.553546395 -2163666.8747638...
4,0107000-01073012704,113,(POLYGON ((-2884976.178560413 -2164988.8798753...


In [53]:
def createIndex(shapefile):
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(2263))
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findZone(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if zones.geometry[idx].is_valid and zones.geometry[idx].contains(p):
            if zones.plctrpop10[idx]>0:
                return zones.plctract10[idx],zones.plctrpop10[idx]
    return None

def processTweets(pid,_,records):
    import pyproj
    import shapely.geometry as geom
    
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)
    index, zones = createIndex('500cities_tracts.geojson')
    counts = {}

    for row in records:
        lnglat, tweet = row
        p = geom.Point(proj(lnglat[0], lnglat[1]))
        zone = findZone(p, index, zones)
        if zone:
            if zone[1]>0:
                counts[zone] = counts.get(zone, 0) + 1
    return counts.items()
shapefile = sc.broadcast(cities)
output = drugTweets.mapPartitionsWithIndex(partial(processTweets, shapefile)) \
                    .reduceByKey(lambda x,y: x+y).collect()
                   # .map(lambda x:(x[0][0],float(x[1])/x[0][1]))\
                    # .sortBy(lambda x: x[0]) \
                     #.collect()
                    
output

[(('0651182-06059062800', 4220), 1),
 (('3128000-31109003003', 4947), 1),
 (('3665508-36093020700', 5513), 1),
 (('0652526-06037554600', 3828), 1),
 (('0404720-04013082027', 3978), 1),
 (('2567000-25013801601', 5067), 1),
 (('1253000-12095014608', 4445), 1),
 (('3651000-36085027302', 3667), 1),
 (('3702140-37021000400', 3681), 1),
 (('0644000-06037119700', 5045), 1),
 (('2758000-27123036700', 4222), 1),
 (('0427400-04013422606', 9135), 1),
 (('1714000-17031081500', 4379), 1),
 (('2079000-20173008300', 4042), 1)]

In [54]:
output = drugTweets.mapPartitionsWithIndex(partial(processTweets, shapefile)) \
                    .reduceByKey(lambda x,y: x+y)\
                    .map(lambda x:(x[0][0],round(x[1]/x[0][1],2))).collect()
                   # .sortBy(lambda x: x[0]) \
                    # .collect()
                    
output

[('0651182-06059062800', 0.0),
 ('3128000-31109003003', 0.0),
 ('3665508-36093020700', 0.0),
 ('0652526-06037554600', 0.0),
 ('0404720-04013082027', 0.0),
 ('2567000-25013801601', 0.0),
 ('1253000-12095014608', 0.0),
 ('3651000-36085027302', 0.0),
 ('3702140-37021000400', 0.0),
 ('0644000-06037119700', 0.0),
 ('2758000-27123036700', 0.0),
 ('0427400-04013422606', 0.0),
 ('1714000-17031081500', 0.0),
 ('2079000-20173008300', 0.0)]

In [55]:
zones[zones['plctract10'] == '0137000-01089002921']

,plctract10,plctrpop10,geometry
4943,0137000-01089002921,6224,(POLYGON ((-2802449.485140256 -1764186.8225311...
